In [1]:
# To run this Notebook you need to clone the Bmad-X repository and install it
# as edditable!

In [2]:
import ARESlatticeStage3v1_9 as ares
import bmadx
import cheetah
import matplotlib.pyplot as plt
import ocelot
import torch
from ocelot.adaptors.astra2ocelot import astraBeam2particleArray

/opt/homebrew/Caskroom/miniforge/base/envs/cheetah-demos/lib/python3.9/site-packages/ocelot/cpbd/beam.py:1567: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  slice_analysis = slice_analysis_py if not nb_flag else nb.jit(slice_analysis_py)
/opt/homebrew/Caskroom/miniforge/base/envs/cheetah-demos/lib/python3.9/site-packages/ocelot/cpbd/tm_utils.py:153: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behavi

initializing ocelot...
import: module PYFFTW is not installed. Install it to speed up calculation


In [3]:
particle_array = astraBeam2particleArray("ACHIP_EA1_2021.1351.001")
beam = bmadx.Particle(
    *particle_array.rparticles, 0.0, particle_array.p0c * 1e9, bmadx.M_ELECTRON
)
beam

Astra to Ocelot: charge =  5.000000000001694e-13  C
Astra to Ocelot: particles number =  100000
Astra to Ocelot: energy =  0.10731590244355117  GeV
Astra to Ocelot: s pos =  13.50890653269  m


Particle(x=array([ 5.08957129e-08,  6.15034578e-05,  2.30254514e-08, ...,
        2.08621251e-04,  5.64747744e-05, -6.26614324e-05]), px=array([ 5.99166607e-09,  6.28096994e-07,  6.38793974e-09, ...,
        5.50625646e-06,  1.21757376e-06, -2.37840015e-06]), y=array([-5.47305446e-07, -5.50215348e-07,  4.40441298e-05, ...,
        2.01889701e-04,  2.77884048e-04,  2.16434721e-04]), py=array([-1.22334631e-08, -1.17054384e-08,  1.65533365e-07, ...,
        6.02466623e-06,  7.84711679e-06,  6.54106916e-06]), z=array([-0.00000000e+00,  2.18696679e-06,  6.85092638e-07, ...,
        8.31485105e-07,  1.18904531e-06,  6.57932034e-06]), pz=array([ 0.        , -0.00102566,  0.0011856 , ..., -0.00054731,
        0.00143682, -0.00181577]), s=0.0, p0c=107314685.84229998, mc2=510998.94999999995)

In [4]:
segment = cheetah.Segment.from_ocelot(ares.cell, warnings=False).subcell(
    "AREASOLA1", "AREABSCR1"
)
segment

Segment(elements=ModuleList(
  (0): Marker(name='AREASOLA1')
  (1): Drift(length=tensor(0.1750))
  (2): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM1')
  (3): Drift(length=tensor(0.4280))
  (4): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM2')
  (5): Drift(length=tensor(0.2040))
  (6): VerticalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCVM1')
  (7): Drift(length=tensor(0.2040))
  (8): Quadrupole(length=tensor(0.1220), k1=tensor(0.), misalignment=tensor([0., 0.]), tilt=tensor(0.), name='AREAMQZM3')
  (9): Drift(length=tensor(0.1790))
  (10): HorizontalCorrector(length=tensor(0.0200), angle=tensor(0.), name='AREAMCHM1')
  (11): Drift(length=tensor(0.4500))
  (12): Screen(resolution=tensor([2448, 2040]), pixel_size=tensor([3.5488e-06, 2.5003e-06]), binning=tensor(1), misalignment=tensor([0., 0.]), is_active=False, name='AREABSCR1')
), name

In [5]:
lattice = [
    bmadx.Drift(L=0.1750),
    bmadx.Quadrupole(L=0.1220, K1=10.0),  # AREAMQZM1
    bmadx.Drift(L=0.4280),
    bmadx.Quadrupole(L=0.1220, K1=-9.0),  # AREAMQZM2
    bmadx.Drift(L=0.2040),
    bmadx.SBend(L=0.0200, G=1e-3 / 0.0200, P0C=beam.p0c),  # AREAMCVM1
    bmadx.Drift(L=0.2040),
    bmadx.Quadrupole(L=0.1220, K1=-8.0),  # AREAMQZM3
    bmadx.Drift(L=0.1790),
    bmadx.SBend(L=0.0200, G=-2e-3 / 0.0200, P0C=beam.p0c),  # AREAMCHM1
    bmadx.Drift(L=0.4500),
]

In [6]:
%%timeit
out_beam = bmadx.track_lattice(beam, lattice)

40.5 ms ± 622 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
out_beam = bmadx.track_lattice(beam, lattice)
out_beam

Particle(x=array([-1.66567941e-08, -4.32792107e-05,  3.09825543e-07, ...,
       -1.34115013e-04, -3.64439013e-05,  3.75587533e-05]), px=array([-2.86337034e-08, -5.27421538e-05, -8.64738075e-07, ...,
       -1.71245522e-04, -4.80505413e-05,  5.09825009e-05]), y=array([-2.97942814e-08, -2.85989616e-08,  2.04185582e-06, ...,
        1.16189435e-05,  1.67532978e-05,  1.19980789e-05]), py=array([ 1.03843481e-06,  1.04389640e-06, -8.28973903e-05, ...,
       -3.84308696e-04, -5.28226243e-04, -4.12258560e-04]), z=array([-2.00071902e-11,  2.08998838e-06,  7.38066400e-07, ...,
        5.58836216e-07,  1.08082523e-06,  6.44747581e-06]), pz=array([ 0.        , -0.00102566,  0.0011856 , ..., -0.00054731,
        0.00143682, -0.00181577]), s=2.046, p0c=107314685.84229998, mc2=510998.94999999995)